In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from scipy.io import loadmat
from pathlib import Path
import matplotlib.patches as patches
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa
from scipy import stats
from scipy.stats import mannwhitneyu, ttest_rel
from scipy.signal import butter, filtfilt
import mne
import eeg_analysis.funcs4eeg as fe
import re
import imp
import ast
import behavior.func4behav as fb
imp.reload(fe)
imp.reload(fb)

from scipy import signal
from scipy.signal import resample
from scipy.ndimage import zoom

In [2]:
case_title = 'endo valid'
case_list = [case_title+' fast', case_title+' slow']
if 'endo' in case_title:
    tmax_list = [1.5+1+0.5+0.05+1, 1.5+1+1+0.05+1]
else:
    tmax_list = [1.5+0.033*4+0.5+0.05+1, 1.5+0.033*4+1+0.05+1]

fs = 1200
for i, case in enumerate(case_list):
    tmax = tmax_list[i]
    watch = '1 fixation'
    tmin = 0 # include fix or not?

    highpass = None # 0.3
    lowpass = None

    EP_lists, RT_lists = fe.pipeline_session_RT(case, watch, tmin, tmax, hipass=highpass, lopass=lowpass, baseline=(0,0), move_baseline=False, detrend=1)

    if 'slow' in case:
        sessions_slow = EP_lists
        rts_slow = RT_lists
    else:
        sessions_fast = EP_lists
        rts_fast = RT_lists


In [3]:
eegs_list = [sessions_fast, sessions_slow]
rts_list = [rts_fast, rts_slow]
session_names = ["sham before", "sham after", "real before", "real after"]
freq_band_names = ["theta", "alpha", "beta", "gamma"]
region_names = ["frontal", "central", "parietal", "occipital", "temporal"] # , "all"]
stage_names_endo = ["fixation", "cue", "wait", "response"]#, "until response"]
stage_names_exo = ["fixation", "wait", "response"]#, "until response"]
freq_bands = [[4, 8], [8, 12], [12, 30], [30, 50]]
endo_fast_stages = [[0, 1.5], [1.5, 1.5+1], [1.5+1, 1.5+1+0.5], [1.5+1+0.5+0.05, 1.5+1+0.5+0.05+0.3]]#, [0, 1.5+1+0.5+0.05]]
endo_slow_stages = [[0, 1.5], [1.5, 1.5+1], [1.5+1, 1.5+1+1], [1.5+1+1+0.05, 1.5+1+1+0.05+0.3]]#, [0, 1.5+1+1+0.05]]
exo_fast_stages = [[0, 1.5], [1.5+0.033*4, 1.5+0.033*4+0.5], [1.5+0.033*4+0.5+0.05, 1.5+0.033*4+0.5+0.05+0.3]]#, [0, 1.5+0.033*4+0.5+0.05]]
exo_slow_stages = [[0, 1.5], [1.5+0.033*4, 1.5+0.033*4+1], [1.5+0.033*4+1+0.05, 1.5+0.033*4+1+0.05+0.3]]#, [0, 1.5+0.033*4+1+0.05]]

if 'endo' in case_title:
    stages_list = [endo_fast_stages, endo_slow_stages]
    stage_names = stage_names_endo
else:
    stages_list = [exo_fast_stages, exo_slow_stages]
    stage_names = stage_names_exo
bp_rt = pd.DataFrame(columns=['stage', 'band', 'region', 'session', 'group_id', 'band power', 'reaction time'])

In [11]:
for i, sessions in enumerate(eegs_list):
    rts = rts_list[i]
    stage_list = stages_list[i]
    for j, session_name in enumerate(session_names):
        session_eeg = sessions[j]
        session_rt = rts[j]
        # print(session_name)
        for group_id in range(9):
            # print(group_id)
            eeg = session_eeg[group_id]
            rt = session_rt[group_id]
            for trial in range(eeg.shape[0]):
                trial_data = eeg[trial,:,:]
                for k, stage in enumerate(stage_list):
                    stage_start = int(stage[0]*fs)
                    stage_end = int(stage[1]*fs)
                    psds, freqs = mne.time_frequency.psd_array_multitaper(trial_data[:,stage_start:stage_end], fs, fmin=3, fmax=51, verbose=False)
                    for l, freq_band in enumerate(freq_bands):
                        bp = fe.band_power(psds, freqs, freq_band)
                        for region in region_names:
                            channels_dict = fe.pick_cortex(region)
                            channel_names_list = list(channels_dict.keys())
                            channels = [channels_dict[key]-1 for key in channel_names_list]
                            bp_region = np.nanmean(bp[channels])
                            new_row = pd.DataFrame({
                                'stage': [stage_names[k]],
                                'band': [freq_band_names[l]],
                                'region': [region],
                                'session': [session_name],
                                'group_id': [group_id],
                                'band power': [bp_region],
                                'reaction time': [rt[trial]]
                            })
                            bp_rt = pd.concat([bp_rt, new_row], ignore_index=True)
bp_rt
# 11 min 32 s for endo or exo
# 42 s for endo invalid
# 3 min 48 s for endo valid

,stage,band,region,session,group_id,band power,reaction time
0,fixation,theta,frontal,sham before,0,4.917636e-09,0.42679
1,fixation,theta,central,sham before,0,6.879493e-09,0.42679
2,fixation,theta,parietal,sham before,0,4.384868e-09,0.42679
3,fixation,theta,occipital,sham before,0,3.601555e-09,0.42679
4,fixation,theta,temporal,sham before,0,4.997879e-09,0.42679
...,...,...,...,...,...,...,...
132155,response,gamma,frontal,real after,8,1.133832e-09,0.27778
132156,response,gamma,central,real after,8,1.103706e-09,0.27778
132157,response,gamma,parietal,real after,8,1.461553e-09,0.27778
132158,response,gamma,occipital,real after,8,1.087505e-09,0.27778


In [12]:
# compute cc for each session
def remove_max_val(arr1, arr2):
    max_idx = np.argmax(arr1)
    arr1 = np.delete(arr1, max_idx)
    arr2 = np.delete(arr2, max_idx)
    return arr1, arr2

def remove_min_val(arr1, arr2):
    max_idx = np.argmin(arr1)
    arr1 = np.delete(arr1, max_idx)
    arr2 = np.delete(arr2, max_idx)
    return arr1, arr2


ccp_session = pd.DataFrame(columns=['stage', 'band', 'region', 'session', 'group id', 'cc', 'p'])
for stage in stage_names:
    for band in freq_band_names:
        for region in region_names:
            for session in session_names:
                for group_id in range(9):
                    roi = bp_rt[(bp_rt['stage']==stage) & (bp_rt['band']==band) & (bp_rt['region']==region) & (bp_rt['session']==session) & (bp_rt['group_id']==group_id)]
                    bps = roi['band power'].values
                    rts = roi['reaction time'].values
                    
                    bps, rts = remove_max_val(bps, rts)
                    bps, rts = remove_min_val(bps, rts)
                    rts, bps = remove_max_val(rts, bps)
                    rts, bps = remove_min_val(rts, bps)

                    cc, p = stats.pearsonr(bps, rts)

                    new_row = pd.DataFrame({
                        'stage': [stage],
                        'band': [band],
                        'region': [region],
                        'session': [session],
                        'group id': [group_id],
                        'cc': [cc],
                        'p': [p]
                    })
                    ccp_session = pd.concat([ccp_session, new_row], ignore_index=True)

ccp_session

,stage,band,region,session,group id,cc,p
0,fixation,theta,frontal,sham before,0,0.026257,0.868898
1,fixation,theta,frontal,sham before,1,0.451884,0.002075
2,fixation,theta,frontal,sham before,2,0.066041,0.670165
3,fixation,theta,frontal,sham before,3,0.155056,0.320793
4,fixation,theta,frontal,sham before,4,-0.049549,0.758354
...,...,...,...,...,...,...,...
2875,response,gamma,temporal,real after,4,-0.083313,0.595318
2876,response,gamma,temporal,real after,5,-0.039827,0.804733
2877,response,gamma,temporal,real after,6,0.391274,0.008631
2878,response,gamma,temporal,real after,7,-0.042734,0.793456


In [24]:
ccp_session.sort_values(by=['cc'], ascending=True).head(10)

,stage,band,region,session,group id,cc,p
1013,cue,alpha,occipital,sham before,5,-0.548910,0.003027
1193,cue,beta,occipital,sham before,5,-0.518336,0.005610
977,cue,alpha,parietal,sham before,5,-0.515098,0.005969
905,cue,alpha,frontal,sham before,5,-0.471080,0.013130
941,cue,alpha,central,sham before,5,-0.470176,0.013331
2124,wait,gamma,temporal,sham before,0,-0.445852,0.003084
2100,wait,gamma,occipital,sham after,3,-0.438187,0.004164
963,cue,alpha,central,real after,0,-0.436216,0.003878
709,fixation,gamma,temporal,real before,7,-0.436152,0.003459
1368,cue,gamma,occipital,sham before,0,-0.432094,0.004269


In [38]:
# average by group, one 'sham before' one row
ccp_table = pd.DataFrame(columns=['stage', 'band', 'region', 'session', 'cc'])
for stage in stage_names:
    for band in freq_band_names:
        for region in region_names:
            for session in session_names:
                roi = ccp_session[(ccp_session['stage']==stage) & (ccp_session['band']==band) & (ccp_session['region']==region) & (ccp_session['session']==session)]
                ccs = roi['cc'].values
                ccs_rounded = np.around(ccs, 2)
                ccs_rounded = np.sort(ccs_rounded)
                # ccs_rounded = ccs_rounded[::-1]

                new_row = pd.DataFrame({
                    'stage': [stage],
                    'band': [band],
                    'region': [region],
                    'session': [session],
                    'cc': [str(ccs_rounded)],  # Write the rounded values to ccp_table
                })
                ccp_table = pd.concat([ccp_table, new_row], ignore_index=True)

ccp_table.sort_values(by=['cc'], ascending=False).head(10)

,stage,band,region,session,cc
112,cue,alpha,occipital,sham before,[-0.55 -0.29 -0.08 -0.04 -0.03 -0.02 -0.02 0....
108,cue,alpha,parietal,sham before,[-0.52 -0.33 -0.1 -0.06 -0.06 0.15 0.18 0....
132,cue,beta,occipital,sham before,[-0.52 -0.3 -0.15 -0.15 -0.12 -0.02 0.01 0....
104,cue,alpha,central,sham before,[-0.47 -0.25 -0.07 -0.05 -0.02 0.11 0.15 0....
100,cue,alpha,frontal,sham before,[-0.47 -0.15 -0.15 -0.11 -0.07 -0.06 0.02 0....
236,wait,gamma,temporal,sham before,[-0.45 -0.22 -0.07 -0.02 0.06 0.08 0.14 0....
78,fixation,gamma,temporal,real before,[-0.44 -0.25 -0.24 -0.13 -0.11 -0.09 -0.04 0....
107,cue,alpha,central,real after,[-0.44 -0.23 -0.2 -0.17 -0.02 0.09 0.11 0....
233,wait,gamma,occipital,sham after,[-0.44 -0.17 -0.11 -0.08 -0.04 -0. 0.07 0....
152,cue,gamma,occipital,sham before,[-0.43 -0.17 -0.11 0.04 0.07 0.13 0.21 0....


In [13]:
stage = "cue"
band = "gamma"
region = "temporal"
bp_rt_roi = bp_rt_session[(bp_rt_session['stage']==stage) & (bp_rt_session['band']==band) & (bp_rt_session['region']==region)]
bps, rts = bp_rt_roi['band power'].values, bp_rt_roi['reaction time'].values
bps, rts = remove_outliers(case_title, bps, rts)
matched_rows = pd.DataFrame()

for bp_value, rt_value in zip(bps, rts):
    mask = (bp_rt_roi['band power'] == bp_value) & (bp_rt_roi['reaction time'] == rt_value)
    matched_rows = pd.concat([matched_rows, bp_rt_roi[mask]])


bp_sb, rt_sb = matched_rows[matched_rows['session']=='sham before']['band power'].values, matched_rows[matched_rows['session']=='sham before']['reaction time'].values
bp_sa, rt_sa = matched_rows[matched_rows['session']=='sham after']['band power'].values, matched_rows[matched_rows['session']=='sham after']['reaction time'].values
bp_rb, rt_rb = matched_rows[matched_rows['session']=='real before']['band power'].values, matched_rows[matched_rows['session']=='real before']['reaction time'].values
bp_ra, rt_ra = matched_rows[matched_rows['session']=='real after']['band power'].values, matched_rows[matched_rows['session']=='real after']['reaction time'].values
colors = ['lightblue', 'blue', 'lightcoral', 'red']


# Concatenate all the updated band power and reaction time arrays for regression
bp = np.concatenate((bp_sb, bp_sa, bp_rb, bp_ra))
rt = np.concatenate((rt_sb, rt_sa, rt_rb, rt_ra))

# Recalculate the regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(bp, rt)
line = slope*bp+intercept

# Update the plot
plt.figure(figsize=(5,5))
plt.plot(bp, line, 'k', label=' r='+str(round(r_value, 3))+', p='+str(round(p_value, 3)))

plt.scatter(bp_sb, rt_sb, label='sham before', color=colors[0])
plt.scatter(bp_sa, rt_sa, label='sham after', color=colors[1])
plt.scatter(bp_rb, rt_rb, label='real before', color=colors[2])
plt.scatter(bp_ra, rt_ra, label='real after', color=colors[3])

plt.legend()
plt.title(case_title+': '+band+', '+region+', '+stage)
plt.xlabel("band power")
plt.ylabel("reaction time")
plt.show()

NameError: name 'bp_rt_session' is not defined